In [173]:
include("quantum_code.jl")
include("decoder.jl")
using BenchmarkTools

In [22]:
ccode = read_code("ccodes/16_12_3_4.txt")

ClassicalCode(16, 12, 3, 4, Integer[6 8 11; 1 8 12; … ; 4 7 9; 2 4 5], Integer[2 8 9 13; 5 6 8 16; … ; 1 9 10 11; 2 5 7 11])

In [80]:
error = random_error(ccode, 0.5)
@time compute_synd_matrix(ccode, error)

  0.000070 seconds (426 allocations: 13.406 KiB)


16×12 BitMatrix:
 0  1  0  1  0  0  0  1  1  1  0  1
 1  0  0  0  1  0  0  1  1  0  1  1
 0  0  0  1  1  0  1  0  1  0  0  1
 1  0  1  0  0  1  0  1  1  0  0  1
 1  0  0  1  0  0  1  0  0  0  0  1
 0  1  0  0  1  0  0  0  0  0  0  0
 1  1  0  0  1  1  1  1  1  1  1  1
 0  1  0  1  0  0  0  0  0  0  0  0
 1  0  1  0  0  1  0  0  0  1  0  0
 0  0  1  1  1  1  0  0  0  1  0  0
 1  1  1  1  0  1  0  0  1  0  1  0
 1  0  1  0  0  0  0  0  0  0  0  0
 1  1  0  0  1  1  0  0  0  0  0  0
 1  0  1  1  0  0  0  0  0  1  0  0
 0  1  0  0  1  1  0  0  0  0  0  0
 1  1  1  1  1  1  0  0  0  0  0  0

In [88]:
synd = compute_synd_matrix(ccode, error)
table = LookupTable(ccode, synd, 0, [], [], 0)
synd_gen = [
    [false, false, false],
    [false, false, true],
    [false, false, true],
    [true, false, true]
]

4-element Vector{Vector{Bool}}:
 [0, 0, 0]
 [0, 0, 1]
 [0, 0, 1]
 [1, 0, 1]

In [143]:
gen = (6,3)
c1, v2 = gen
ver = table.ccode.check_nbhd[c1, :]
hor = table.ccode.bit_nbhd[v2, :]
synd_gen = falses(table.ccode.dc, table.ccode.dv)

for i = 1:table.ccode.dc
    for j = 1:table.ccode.dv
        if (table.synd_matrix[ver[i], hor[j]])
            synd_gen[i, j] = true
        end
    end
end

synd_gen

4×3 BitMatrix:
 0  0  1
 0  0  1
 1  1  1
 0  0  0

In [184]:
include("decoder.jl")


update (generic function with 1 method)

In [185]:
@btime score_gen(table, synd_gen)

  13.400 μs (186 allocations: 7.41 KiB)


(3, 1, ([3], Any[]))